In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import pydicom

##Import any other stats/DL/ML packages you may need here. E.g. Keras, scikit-learn, etc.
from cv2 import cv2

import skimage

import tensorflow as tf
import tensorflow.keras.layers

# Inference


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filenames):
    dicom_data = []
    for img_name in filenames:
        print('Load file {} ...'.format(img_name))
        dcm = pydicom.dcmread(img_name)
        fields = [dcm.PatientID, int(dcm.PatientAge), dcm.PatientSex, dcm.Modality, dcm.StudyDescription, dcm.Rows, dcm.Columns, img_name, dcm.pixel_array]
        dicom_data.append(fields)
    dicom_df = pd.DataFrame(dicom_data, columns = ['PatientID','PatientAge','PatientSex','Modality','Finding Labels','Rows','Columns', 'Filename', 'Image'])
    print(dicom_df)
    return dicom_df
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def clahe_scaling(image):
        clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(16, 16))
        return clahe.apply(image)

def preprocess_image(src, IMG_SIZE=(224,224), INPUT_SHAPE=(1, 224,224,3)):
#     '''Takes an input image and returns a modified version of it'''
    dst = cv2.resize(src, IMG_SIZE) / 255
    dst = skimage.img_as_ubyte(dst)
    dst = clahe_scaling(dst)
    dst = skimage.img_as_float(dst)
    dst = cv2.cvtColor(dst.astype('float32'), cv2.COLOR_GRAY2RGB ) * 255
    return np.broadcast_to(dst, INPUT_SHAPE)

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # load model
    model = tensorflow.keras.models.load_model(model_path)
    model.load_weights(weight_path)
    # summarize model.
    print(model.summary())
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction = model.predict(img, verbose = True)
    return prediction < thresh

In [3]:
my_model = load_model(model_path='my_model.h5', weight_path='fine_tunning.best.hdf5')

2021-11-10 17:06:52.873096: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 17:06:52.897133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 17:06:52.898062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 17:06:52.900026: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
VGG16_preprocessing (Sequent (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
Flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25088)             0         
_________________________________________________________________
FC1 (Dense)                  (None, 256)               6422784   
_________________________________________________________________
prediction (Dense)           (None, 1)                 257   

### Checking DICOM test files

In [4]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
dicom_df = check_dicom(test_dicoms)



IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

# - ImgNet mean subtraction preprocessing is already into the model
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

Load file test1.dcm ...
Load file test2.dcm ...
Load file test3.dcm ...
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...
  PatientID  PatientAge PatientSex Modality Finding Labels  Rows  Columns  \
0         2          81          M       DX     No Finding  1024     1024   
1         1          58          M       DX   Cardiomegaly  1024     1024   
2        61          77          M       DX       Effusion  1024     1024   
3         2          81          M       DX     No Finding  1024     1024   
4         2          81          M       CT     No Finding  1024     1024   
5         2          81          M       DX     No Finding  1024     1024   

    Filename                                              Image  
0  test1.dcm  [[199, 175, 152, 133, 124, 118, 113, 111, 110,...  
1  test2.dcm  [[202, 199, 195, 193, 195, 194, 193, 192, 184,...  
2  test3.dcm  [[142, 142, 143, 141, 143, 140, 140, 136, 137,...  
3  test4.dcm  [[199, 175, 152, 133, 124, 118, 113, 1

### Testing Predictions

In [5]:
thresh = 0.24

# use the .dcm files to test your prediction
for index, img_data in dicom_df.iterrows():
    img = img_data['Image']
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img)
    pred = predict_image(my_model,img_proc,thresh)
    print(f'Predicted: {pred}, findings: {img_data["Finding Labels"]}')

2021-11-10 17:06:53.832133: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-10 17:06:54.162729: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2021-11-10 17:06:54.339616: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 2s 2s/step
Predicted: [[False]], findings: No Finding
1/1 [==============================] - 0s 11ms/step
Predicted: [[ True]], findings: Cardiomegaly
1/1 [==============================] - 0s 11ms/step
Predicted: [[False]], findings: Effusion
1/1 [==============================] - 0s 10ms/step
Predicted: [[False]], findings: No Finding
1/1 [==============================] - 0s 11ms/step
Predicted: [[False]], findings: No Finding
1/1 [==============================] - 0s 11ms/step
Predicted: [[False]], findings: No Finding
